In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns  # visualization tool

from sklearn.datasets import load_diabetes
diabetes = load_diabetes()

pd.DataFrame(diabetes.data, columns=("age", "sex", "bmi", "map", "tc", "ldl", "hdl", "tch", "ltg", "glu"))




In [ ]:
diabetes_df = pd.read_table('diabetes.tab.csv', nrows=None)
print('diabetes_df shape:', diabetes_df.shape)
diabetes_df = diabetes_df.rename(columns={'BP':'map','S1': 'tc','S2':'ldl','S3':'hdl','S4':'tch','S5':'ltg','S6':'glu'})

diabetes_df.head()


In [ ]:
diabetes_df.describe()


In [ ]:
corrmat = diabetes_df.corr()
f, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(corrmat, vmax=.8, square=True);


In [ ]:
k  = 11
cols = corrmat.nlargest(k, 'Y')['Y'].index
cm = np.corrcoef(diabetes_df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar = True, annot = True, square=True, fmt='.2f', annot_kws={'size':10},yticklabels=cols.values,xticklabels=cols.values)
plt.show()


In [ ]:


from sklearn import linear_model
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns  # visualization tool

from sklearn.datasets import load_diabetes
diabetes = load_diabetes()

diabetes_df = pd.DataFrame(diabetes.data, columns=("age", "sex", "bmi", "map", "tc", "ldl", "hdl", "tch", "ltg", "glu"))



linear_regression = linear_model.LinearRegression()

#説明変数を縦(1)の列と指定して削除します！
X = diabetes_df.copy()

#Yに目的変数を入れます！
Y = diabetes.target


In [ ]:
import sklearn.model_selection

#左辺にモデル構築用のXY,テスト用のX,Yを用意します。
#先ほど作成した、説明変数を第一引数に、目的変数を第二引数に置きます。
#train_test_splitを足すことを忘れないでください！！
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X,Y)


# 学習用とテスト用でデータを分離
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1,random_state=0)


In [ ]:
# リッジ回帰アルゴリズム
ridgereg = linear_model.Ridge()
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y)
ridgereg.fit(X_train, y_train)
print("train mean_squared_error",metrics.mean_squared_error(Y_train, ridgereg.predict(X_train)))
print("score",ridgereg.score(X_train,y_train))
y_pred = ridgereg.predict(X_test)
print("test mean_squared_error",metrics.mean_squared_error(Y_test, y_pred))
mse = metrics.mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)
print("rmse",rmse)




In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import lightgbm as lgb


estimator = lgb.LGBMRegressor(objective='regression_l2', learning_rate = 0.2, n_estimators = 100, boosting='gbdt',max_bin=31,min_data=1)

params = {
    'num_leaves': [x for x in range(30, 70, 10)],
    'metric': ('l1', 'l2')
    }
gridsearch = GridSearchCV(estimator, params)

# 変換器・推定器オブジェクト作成
standardizer = StandardScaler()
pca = PCA(n_components=2)

pl = Pipeline([
    ('standardize', standardizer),
    ('pca', pca),
    ('clf', gridsearch)
    ])



pl.fit(X_train, y_train)

# テストデータに対して予測
prediction = pl.predict(X_test)

# 評価
mse = mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)
print("rmse",rmse)
